<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#0.-해결하는-문제" data-toc-modified-id="0.-해결하는-문제-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>0. 해결하는 문제</a></span><ul class="toc-item"><li><span><a href="#샘플-코드-확인" data-toc-modified-id="샘플-코드-확인-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>샘플 코드 확인</a></span></li></ul></li><li><span><a href="#1.-데이터-로드" data-toc-modified-id="1.-데이터-로드-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. 데이터 로드</a></span></li><li><span><a href="#간단한-EDA" data-toc-modified-id="간단한-EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>간단한 EDA</a></span></li><li><span><a href="#데이터-로드-->-날짜,자기상관-변수-Feature-Engineering-자동화-함수" data-toc-modified-id="데이터-로드-->-날짜,자기상관-변수-Feature-Engineering-자동화-함수-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>데이터 로드 -&gt; 날짜,자기상관 변수 Feature Engineering 자동화 함수</a></span></li><li><span><a href="#자기상관-변수-FE" data-toc-modified-id="자기상관-변수-FE-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>자기상관 변수 FE</a></span></li></ul></div>

## 0. 해결하는 문제
- 가스공급량을 한도 90일 내에 예측하는 문제
- 메트릭 : <a href='https://dacon.io/competitions/official/235830/overview/rules'>NMAE</a>
- 저작권이 허용 가능한 외부 데이터 및 사전학습 모델 사용 가능

### 샘플 코드 확인

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

# Test 데이터
test_df = pd.read_csv('test.csv')
test_df.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [2]:
# 샘플 제출 데이터
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0
...,...,...
15115,2019-03-31 20 H,0
15116,2019-03-31 21 H,0
15117,2019-03-31 22 H,0
15118,2019-03-31 23 H,0


## 1. 데이터 로드

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')
print(df.shape)

(368088, 4)


In [4]:
# 결측치 확인 -> 없음
df.isnull().sum()

연월일    0
시간     0
구분     0
공급량    0
dtype: int64

In [5]:
# 연월일, 시간, 구분의 unique한 값들 출력
cols = ['연월일', '시간', '구분']
for col in cols:
    display(df[col].unique())

array(['2013-01-01', '2013-01-02', '2013-01-03', ..., '2018-12-29',
       '2018-12-30', '2018-12-31'], dtype=object)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [6]:
# 연월일에서 빈 시간이 없는지 확인 -> 모두 있음
origin_len = len(pd.date_range(start='2013-01-01', end='2018-12-31', freq='D'))
df_len = len(df['연월일'].unique())

print('원래 날짜 길이:', origin_len, '데이터프레임 속 날짜 길이:', df_len)

원래 날짜 길이: 2191 데이터프레임 속 날짜 길이: 2191


## 간단한 EDA
- 공급량(Y) 분포 확인

In [7]:
plot_df = df.copy()

In [8]:
plot_df['연월일'] = pd.to_datetime(plot_df['연월일'])

In [14]:
def add_datetime(row):
    if row['시간'] == 24:
        row['연월일'] = row['연월일'] + pd.DateOffset(days=1)
        row['시간'] = 0
    return row

In [15]:
plot_df = plot_df.apply(add_datetime, axis=1)

In [16]:
plot_df['시간'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23,  0])

In [17]:
plot_df.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


In [18]:
plot_df['연월일'] = plot_df['연월일'].astype(str)
plot_df['시간'] = plot_df['시간'].apply(lambda x: str(x) if x >= 10 else '0'+str(x))
plot_df['datetime'] = plot_df['연월일'] + ' ' + plot_df['시간']
plot_df['datetime'] = pd.to_datetime(plot_df['datetime'], format='%Y-%m-%d %H')

In [26]:
def show_displot(data, x, y, hue, kind):
    plt.figure(figsize=(22, 10))
    sns.displot(data=data, x=x, y=y, hue=hue, kind=kind)
    plt.xticks(rotation=90)
    plt.show()

## 데이터 로드 -> 날짜,자기상관 변수 Feature Engineering 자동화 함수

In [101]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings(action='ignore')


def load_data() -> pd.DataFrame:
    path = '/Users/younghun/Desktop/gitrepo/dacon_gas'

    data = pd.read_csv(os.path.join(path, '한국가스공사_시간별 공급량_20181231.csv'),
                       encoding='cp949')
    return data


def make_datetime_vars(data) -> pd.DataFrame:
    data['연월일'] = pd.to_datetime(data['연월일'])
    data['year'] = data['연월일'].dt.year
    data['month'] = data['연월일'].dt.month
    data['day'] = data['연월일'].dt.day
    data['week_no'] = data['연월일'].dt.strftime("%V").astype(int)
    data['dayofweek'] = data['연월일'].dt.day_name()
    data['weekend_yn'] = np.where(data['dayofweek'].isin(['Saturday', 'Sunday']), 1, 0)
    data['dayofyear'] = data['연월일'].dt.dayofyear

    return data


def make_autocorel_vars(data, gubun='A') -> pd.DataFrame:
    sub_df = data[data['구분'] == gubun]
    # 자기 상관 변수 생성
    n = [x for x in range(1, 6)]
    for i in n:
        column_nm = f'D-{i}_y'
        sub_df[column_nm] = sub_df['공급량'].shift(i)
    # 이동평균 변수 생성
    n = [5, 7, 14, 30]
    for i in n:
        column_nm = f'D-{i}_ma_avg'
        sub_df[column_nm] = sub_df['공급량'].rolling(i).mean().shift(1)
    # 증가율 변수 생성
    sub_df['D1-D2_ratio'] = (sub_df['D-1_y'] - sub_df['D-2_y']) / sub_df['D-2_y']
    sub_df['W1-W2_ratio'] = (sub_df['D-7_ma_avg'] - sub_df['D-14_ma_avg']) / sub_df['D-14_ma_avg']
    sub_df['D1-W1_ratio'] = (sub_df['D-1_y'] - sub_df['D-7_ma_avg']) / sub_df['D-7_ma_avg']
    sub_df['D1-W2_ratio'] = (sub_df['D-1_y'] - sub_df['D-14_ma_avg']) / sub_df['D-14_ma_avg']
    
    return sub_df


def split_and_merge_datasets():
    data = load_data()
    # 날짜 관련 변수 생성
    data = make_datetime_vars(data)
    # 모든 구분에 대해서 분리해서 자기상관변수 생성 후 다시 합치기
    merge_df = pd.DataFrame()
    cols = data['구분'].unique()
    for col in cols:
        sub_df = make_autocorel_vars(data, gubun=col)
        merge_df = pd.concat([merge_df, sub_df], axis=0)
    
    return merge_df

merge_df = split_and_merge_datasets()
print(merge_df.shape)

(368088, 24)


---

## 자기상관 변수 FE

In [148]:
# Train, Test 병합
test_df['연월일'] = test_df['일자|시간|구분'].str.split(' ', expand=True)[0]
test_df['시간'] = test_df['일자|시간|구분'].str.split(' ', expand=True)[1].astype(int)
test_df['구분'] = test_df['일자|시간|구분'].str.split(' ', expand=True)[2]

del test_df['일자|시간|구분']

df = pd.concat([df, test_df], axis=0)

In [149]:
df

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105
...,...,...,...,...
15115,2019-03-31,20,H,NaN
15116,2019-03-31,21,H,NaN
15117,2019-03-31,22,H,NaN
15118,2019-03-31,23,H,NaN


In [150]:
df['연월일'] = pd.to_datetime(df['연월일'])

In [151]:
def change_dates(row):
    if row['시간'] == 24:
        row['연월일'] += pd.DateOffset(days=1)
        row['시간'] = 0
    return row
df = df.apply(change_dates, axis=1)

In [152]:
df['시간'] = df['시간'].apply(lambda x: str(x) if x >= 10 else '0'+str(x))

In [153]:
df['연월일'] = df['연월일'].astype(str)
df['datetime'] = df['연월일'] + ' ' + df['시간']

In [154]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

# 매년 01-01 ~ 03-31 데이터인 것만 따로 떼네기
date_range = (df['datetime'].dt.month >= 1) & (df['datetime'].dt.month <= 3)
df = df[date_range]

In [155]:
df['prev_datetime'] = df['datetime'] - pd.DateOffset(years=1)

---
- 구분 유형 하나로 파생변수 모두 만든다음 -> 모든 구분 유형에 대해 반복 loop

In [160]:
A_df = df[df['구분'] == 'A']

- 일년, 이년전 그 시간 대 공급량 파생변수

In [161]:
# 일년 전 공급량 파생변수 생성
A_df['prev_year_공급량'] = A_df.groupby([A_df['datetime'].dt.month, A_df['datetime'].dt.day,
                                 A_df['datetime'].dt.hour])['공급량'].shift()

In [162]:
# 이년 전 공급량 파생변수 생성
A_df['prev_two_year_공급량'] = A_df.groupby([A_df['datetime'].dt.month, A_df['datetime'].dt.day,
                                     A_df['datetime'].dt.hour])['공급량'].shift(2)

- 작년도 평균 공급량 파생변수

In [163]:
A_df['prev_year'] = A_df['prev_datetime'].dt.year

In [165]:
prev_year_mean_df = A_df.groupby([A_df['datetime'].dt.year])['공급량'].mean()
prev_year_mean_dict = dict(zip(prev_year_mean_df.index, prev_year_mean_df.values.reshape(-1,)))

In [169]:
A_df['prev_year_avg_공급량'] = A_df['prev_year'].map(prev_year_mean_dict)

- 작년도 월별 평균 공급량 파생변수

In [172]:
prev_month_mean_df = A_df.groupby([A_df['datetime'].dt.year, A_df['datetime'].dt.month])['공급량'].mean()
prev_month_mean_dict = dict(zip(prev_month_mean_df.index, prev_month_mean_df.values.reshape(-1,)))

In [183]:
A_df['prev_year_month'] = tuple(zip(A_df['prev_year'].values.reshape(-1,), A_df['prev_datetime'].dt.month.values.reshape(-1,)))

In [186]:
A_df['prev_month_avg_공급량'] = A_df['prev_year_month'].map(prev_month_mean_dict)

- 작년도 월-일자별 평균 공급량 파생변수

In [189]:
prev_month_day_mean_df = A_df.groupby([A_df['datetime'].dt.year,
                                       A_df['datetime'].dt.month,
                                      A_df['datetime'].dt.day])['공급량'].mean()
prev_month_day_mean_dict = dict(zip(prev_month_day_mean_df.index, prev_month_day_mean_df.values.reshape(-1,)))

A_df['prev_year_month_day'] = tuple(zip(A_df['prev_year'].values.reshape(-1,),
                                       A_df['prev_datetime'].dt.month.values.reshape(-1,),
                                       A_df['prev_datetime'].dt.day.values.reshape(-1,)))

In [192]:
A_df['prev_month_day_avg_공급량'] = A_df['prev_year_month_day'].map(prev_month_day_mean_dict)

- 작년도 월-시간별 평균 공급량 파생변수

In [195]:
prev_month_hour_mean_df = A_df.groupby([A_df['datetime'].dt.year,
                                       A_df['datetime'].dt.month,
                                       A_df['datetime'].dt.hour])['공급량'].mean()
prev_month_hour_mean_dict = dict(zip(prev_month_hour_mean_df.index, prev_month_hour_mean_df.values.reshape(-1,)))

A_df['prev_year_month_hour'] = tuple(zip(A_df['prev_year'].values.reshape(-1,),
                                       A_df['prev_datetime'].dt.month.values.reshape(-1,),
                                       A_df['prev_datetime'].dt.hour.values.reshape(-1,)))

In [197]:
A_df['prev_month_hour_avg_공급량'] = A_df['prev_year_month_hour'].map(prev_month_hour_mean_dict)

In [213]:
used_cols = ['연월일', '시간', 'datetime', '구분', '공급량', 'prev_year_공급량',
            'prev_two_year_공급량', 'prev_year_avg_공급량', 'prev_month_avg_공급량', 'prev_month_day_avg_공급량',
            'prev_month_hour_avg_공급량']
A_final_df = A_df[used_cols]

- 위 과정을 모든 구분에 대해 반복!!! -> Train/Test 병합, 날짜변수 생성, 자기상관 변수 생성하는 것까지 클래스화 시키기
    - 2년 전 공급량하면 너무 기간이 많이 비긴 함..
    - 1년 전만 할 것인지 2년 전까지 할 것인지 성능비교를 통해서 결정

In [10]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')


def load_datasets(path) -> pd.DataFrame:
    train = pd.read_csv(os.path.join(path, '한국가스공사_시간별 공급량_20181231.csv'),
                        encoding='cp949')
    test = pd.read_csv(os.path.join(path, 'test.csv'))
    # test 데이터 전처리
    test['연월일'] = test['일자|시간|구분'].str.split(' ', expand=True)[0]
    test['시간'] = test['일자|시간|구분'].str.split(' ', expand=True)[1].astype(int)
    test['구분'] = test['일자|시간|구분'].str.split(' ', expand=True)[2]
    del test['일자|시간|구분']

    data = pd.concat([train, test], axis=0)
    return data


def make_datetime_vars(data) -> pd.DataFrame:
    data['연월일'] = pd.to_datetime(data['연월일'])
    data['year'] = data['연월일'].dt.year
    data['month'] = data['연월일'].dt.month
    data['day'] = data['연월일'].dt.day
    data['week_no'] = data['연월일'].dt.strftime("%V").astype(int)
    data['dayofweek'] = data['연월일'].dt.day_name()
    data['weekend_yn'] = np.where(data['dayofweek'].isin(['Saturday', 'Sunday']), 1, 0)
    data['dayofyear'] = data['연월일'].dt.dayofyear

    return data


def change_dates(row):
    if row['시간'] == 24:
        row['연월일'] += pd.DateOffset(days=1)
        row['시간'] = 0
    return row


def extract_need_data(df):
    """ 연월일 날짜 포맷 바꾸고 매년 01-01~03-31 기간 데이터만 추출"""
    df = df.apply(change_dates, axis=1)
    df['시간'] = df['시간'].apply(lambda x: str(x) if x >= 10 else '0' + str(x))
    df['연월일'] = df['연월일'].astype(str)
    df['datetime'] = df['연월일'] + ' ' + df['시간']
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

    # 매년 01-01 ~ 03-31 데이터인 것만 따로 떼네기
    date_range = (df['datetime'].dt.month >= 1) & (df['datetime'].dt.month <= 3)
    df = df[date_range]
    df['prev_datetime'] = df['datetime'] - pd.DateOffset(years=1)

    return df


def make_autocorr_vars(df, gubun='A'):
    sub_df = df[df['구분'] == gubun]
    # 일년 전 그 시간대 공급량
    sub_df['prev_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                          sub_df['datetime'].dt.hour])['공급량'].shift()
    # 이년 전 공급량 파생변수 생성
    sub_df['prev_two_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                              sub_df['datetime'].dt.hour])['공급량'].shift(2)
    # 일년 전 평균 공급량
    sub_df['prev_year'] = sub_df['prev_datetime'].dt.year
    prev_year_mean_df = sub_df.groupby([sub_df['datetime'].dt.year])['공급량'].mean()
    prev_year_mean_dict = dict(zip(prev_year_mean_df.index, prev_year_mean_df.values.reshape(-1, )))
    sub_df['prev_year_avg_공급량'] = sub_df['prev_year'].map(prev_year_mean_dict)
    # 일년 전 월별 평균 공급량
    prev_month_mean_df = sub_df.groupby([sub_df['datetime'].dt.year, sub_df['datetime'].dt.month])['공급량'].mean()
    prev_month_mean_dict = dict(zip(prev_month_mean_df.index, prev_month_mean_df.values.reshape(-1, )))
    sub_df['prev_year_month'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                        sub_df['prev_datetime'].dt.month.values.reshape(-1, )))
    sub_df['prev_month_avg_공급량'] = sub_df['prev_year_month'].map(prev_month_mean_dict)
    # 일년 전 월-일자별 평균 공급량
    prev_month_day_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                           sub_df['datetime'].dt.month,
                                           sub_df['datetime'].dt.day])['공급량'].mean()
    prev_month_day_mean_dict = dict(zip(prev_month_day_mean_df.index, prev_month_day_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_day'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.day.values.reshape(-1, )))
    sub_df['prev_month_day_avg_공급량'] = sub_df['prev_year_month_day'].map(prev_month_day_mean_dict)
    # 일년 전 월-시간 별 평균 공급량
    prev_month_hour_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                            sub_df['datetime'].dt.month,
                                            sub_df['datetime'].dt.hour])['공급량'].mean()
    prev_month_hour_mean_dict = dict(zip(prev_month_hour_mean_df.index, prev_month_hour_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_hour'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.hour.values.reshape(-1, )))
    sub_df['prev_month_hour_avg_공급량'] = sub_df['prev_year_month_hour'].map(prev_month_hour_mean_dict)

    # 필요한 칼럼들만 추출
    used_cols = ['연월일', '시간', 'datetime', '구분', '공급량', 'prev_year_공급량',
                 'prev_two_year_공급량', 'prev_year_avg_공급량', 'prev_month_avg_공급량', 'prev_month_day_avg_공급량',
                 'prev_month_hour_avg_공급량']
    sub_df = sub_df[used_cols]
    return sub_df


def fe_autocorr_vars(df):
    final_df = pd.DataFrame()
    gubun_cols = df['구분'].unique()
    for gubun in tqdm(gubun_cols):
        sub_df = make_autocorr_vars(df, gubun=gubun)
        final_df = pd.concat([final_df, sub_df], axis=0)
    return final_df



path = '/Users/younghun/Desktop/gitrepo/dacon_gas'
dataset = load_datasets(path)
dataset = make_datetime_vars(dataset)
dataset = extract_need_data(dataset)
final_df = fe_autocorr_vars(dataset)

100%|██████████| 7/7 [00:01<00:00,  6.67it/s]


In [11]:
final_df

,연월일,시간,datetime,구분,공급량,prev_year_공급량,prev_two_year_공급량,prev_year_avg_공급량,prev_month_avg_공급량,prev_month_day_avg_공급량,prev_month_hour_avg_공급량
0,2013-01-01,01,2013-01-01 01:00:00,A,2497.129,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01,02,2013-01-01 02:00:00,A,2363.265,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01,03,2013-01-01 03:00:00,A,2258.505,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,04,2013-01-01 04:00:00,A,2243.969,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01,05,2013-01-01 05:00:00,A,2344.105,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15114,2019-03-31,19,2019-03-31 19:00:00,H,NaN,219.185,355.468,462.382223,320.340968,196.931083,368.914419
15115,2019-03-31,20,2019-03-31 20:00:00,H,NaN,244.162,382.636,462.382223,320.340968,196.931083,402.174387
15116,2019-03-31,21,2019-03-31 21:00:00,H,NaN,248.059,379.556,462.382223,320.340968,196.931083,416.910097
15117,2019-03-31,22,2019-03-31 22:00:00,H,NaN,231.181,353.684,462.382223,320.340968,196.931083,402.301484
